In [1]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

In [ ]:
(df[pd.isna(df["region_1"]) & pd.isna(df["region_2"])])

In [ ]:
df.info()

In [ ]:
df

In [ ]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(df["designation"].value_counts())

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df["designation"].value_counts()[:100])

In [ ]:
#df[df["designation_tr"] == "reserva" | df["designation_tr"] == "riserva" | df["designation_tr"] == "rèserve"]["designation_tr"] = "reserve"
df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])


In [ ]:
df

In [ ]:
# rimuovo regioni doppie e creo regione unica
df.loc[df["region_1"] == df["region_2"], "region_2"] = ""
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)

In [ ]:
singular_region_complete = set(df["region_complete"])
len(singular_region_complete)

In [ ]:
import geocoder

g = geocoder.arcgis('Redlands, CA')
print(g.latlng)

In [ ]:
import geocoder

singular_region_complete_with_latlong = []
for region in singular_region_complete:
    print(region)
    loc =  geocoder.arcgis(region,maxRows=1)
    print(loc)
    try:
        singular_region_complete_with_latlong.append((region, loc.latlng[0],loc.latlng[1]))   
    except:
        pass

In [ ]:
with open('region_coordinates.csv', 'w') as fp:
    fp.write('\n'.join('%s,%f,%f' % x for x in singular_region_complete_with_latlong))

In [ ]:
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, how="left", on="region_complete")

In [ ]:
df.to_csv("competition_dataset/mine_dev.csv",index=False)

In [ ]:
df = pd.get_dummies(df, columns=["designation_tr","province","variety"])

In [ ]:
df.columns

In [10]:
df = pd.read_csv("competition_dataset/mine_dev.csv")

In [11]:
df

,country,description,designation,province,region_1,region_2,variety,winery,quality,designation_tr,region_complete,lat,long
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0,brut blanc de blancs,Crémant d'Alsace,33.97902,-118.414180
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0,NaN,Paso Robles Central Coast,35.62675,-120.692201
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,NaN,Gewürztraminer,Château Bianca,35.0,NaN,Willamette Valley,45.50012,-122.501480
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0,NaN,NaN,NaN,NaN
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0,natì,Pompeiano,45.43135,9.988320
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120739,Italy,This is a lovely Sauvignon with very character...,NaN,Northeastern Italy,Friuli,NaN,Sauvignon,Plozner,40.0,NaN,Friuli,46.07528,13.079720
120740,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,Northern Spain,Rioja,NaN,Tempranillo Blend,Ramón Bilbao,41.0,reserve,Rioja,-29.41336,-66.855900
120741,Australia,Surprisingly complete and harmonious for a Rie...,NaN,South Australia,Adelaide Hills,NaN,Riesling,Wagtail,26.0,NaN,Adelaide Hills,-34.91118,138.707350
120742,Germany,Broad aromas of ripe yellow fruit and a hint o...,Spätlese,Mosel,NaN,NaN,Riesling,Pamona Cellars,26.0,spätlese,NaN,NaN,NaN


import gmaps
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html

locations = df[['lat', 'long']]
weights = df['quality']
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap_layer)
embed_minimal_html('export.html', views=[fig])

In [ ]:
df["variety"].value_counts()[:50]

In [ ]:
df["province"].value_counts()[:50]

df["region_complete"] = df["region_complete"].replace(r'^\s*$', np.NaN, regex=True)


In [12]:
df = pd.get_dummies(df, columns=["region_complete","variety"])

In [13]:
df.to_csv("competition_dataset/mine_dev_with_dummies.csv",index=False)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("competition_dataset/mine_dev_with_dummies.csv")

In [3]:
len(df.columns)

1824

In [4]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=10, n_jobs=-1,max_features="sqrt")

In [5]:
X = df.loc[:, df.columns != 'quality']
y = df["quality"]

In [6]:
X.shape

(120744, 1823)

In [ ]:
regr.fit(X,y)

In [1]:
regr

NameError: name 'regr' is not defined